In [37]:
from tsf import TSF
import pandas as pd

from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [2]:
historical = pd.read_parquet("data/historical_inventory.parquet")
current = pd.read_parquet("data/current_inventory.parquet")
df = pd.concat([historical, current])
df.to_parquet("data/inventory.parquet")

In [8]:
df = pd.read_parquet("data/inventory.parquet")

In [9]:
data = pd.DataFrame()
for plant in df.plant_id.unique():
    temp = df[df.plant_id == plant]
    temporary = pd.DataFrame()
    for material in temp.material_id.unique():
        temp2 = temp[temp.material_id == material]
        temp2.sort_values(by="inventory_month_end_date", inplace=True)
        temporary = pd.concat([temporary, temp2])
    data = pd.concat([df, temporary])

In [51]:
scores = {}
forecasts = {}
for plant in df.plant_id.unique():
    temp = df[df.plant_id == plant]
    tsf = TSF(temp, "inventory_month_end_date", "inventory", "2024-03-31", "2024-03-31", "2025-03-31", "2025-03-31", "A-MAR", "A", False, product_col="material_id")
    svr = SVR()
    hr = HuberRegressor()
    model = StackingRegressor(estimators=[("svr", svr)], final_estimator=hr, passthrough=True)
    data, model, total_scores, material_scores = tsf.tsf(1, 4, 0, True, False, False)
    scores[plant] = total_scores
    forecasts[plant] = data[data.index >= "2024-03-31"][['inventory', 'forecast']]
scores = pd.DataFrame(scores).transpose()

In [52]:
scores['mae'].mean()

1939.1264142857144

In [19]:
scores['mae'].mean()

1890.2293571428568